# Temperature controllers (heaters and coolers)

PyLabRobot supports the following temperature controllers:

- Opentrons Temperature Module V2

Temperature controllers are controlled by the {class}`~pylabrobot.temperature_controlling.temperature_controller.TemperatureController` class. This class takes a backend as an argument. The backend is responsible for communicating with the scale and is specific to the hardware being used.

The {class}`~pylabrobot.temperature_controlling.opentrons.OpentronsTemperatureModuleV2` is a TemperatureController subclass initialized with a tube rack.

In [1]:
from pylabrobot.temperature_controlling import TemperatureController
from pylabrobot.temperature_controlling.opentrons import OpentronsTemperatureModuleV2
from pylabrobot.temperature_controlling.opentrons_backend import (
  OpentronsTemperatureModuleBackend,
)

Using the Opentrons temperature controller currently requires an Opentrons robot. The robot must be connected to the host computer and to the temperature module.

In [2]:
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends.opentrons_backend import OpentronsBackend
from pylabrobot.resources.opentrons import OTDeck

ot = OpentronsBackend(host="169.254.184.185", port=31950)  # Get the ip from the Opentrons app
lh = LiquidHandler(backend=ot, deck=OTDeck())
await lh.setup()

After setting up the robot, use the `OpentronsBackend.list_connected_modules()` to list the connected temperature modules. You are looking for the `'id'` of the module you want to use.

In [3]:
await ot.list_connected_modules()

[{'id': 'fc409cc91770129af8eb0a01724c56cb052b306a',
  'serialNumber': 'TDV21P20201224B13',
  'firmwareVersion': 'v2.1.0',
  'hardwareRevision': 'temp_deck_v21',
  'hasAvailableUpdate': False,
  'moduleType': 'temperatureModuleType',
  'moduleModel': 'temperatureModuleV2',
  'data': {'status': 'idle', 'currentTemperature': 34.0},
  'usbPort': {'port': 1,
   'portGroup': 'main',
   'hub': False,
   'path': '1.0/tty/ttyACM0/dev'}}]

Initialize the OpentronsTemperatureModuleV2 with the `id` of the module you want to use.

In [4]:
t = OpentronsTemperatureModuleV2(name="t", opentrons_id="fc409cc91770129af8eb0a01724c56cb052b306a")
await t.setup()

The `OpentronsTemperatureModuleV2` is a subclass of {class}`~pylabrobot.temperature_controlling.temperature_controller.TemperatureController`.

In [5]:
isinstance(t, TemperatureController)

True

Be sure to assign the temperature controller to the robot deck before you use it. This is done with the usual {func}`~pylabrobot.resources.opentrons.deck.assign_child_at_slot` function.

In [6]:
lh.deck.assign_child_at_slot(t, slot=3)

You can set the temperature in Celsius using {func}`~pylabrobot.temperature_controlling.temperature_controller.TemperatureController.set_temperature`.

In [7]:
await t.set_temperature(37)

Use {func}`~pylabrobot.temperature_controlling.temperature_controller.TemperatureController.wait_for_temperature` to wait for the temperature to stabilize at the target temperature.

In [8]:
await t.wait_for_temperature()

The temperature can be read using {func}`~pylabrobot.temperature_controlling.temperature_controller.TemperatureController.get_temperature`.

In [9]:
await t.get_temperature()

37.0

If you are done with the temperature controller, you can use {func}`~pylabrobot.temperature_controlling.temperature_controller.TemperatureController.deactivate` to turn it off. The temperature controller will return to ambient temperature.

In [10]:
await t.deactivate()

## Pipetting from the OT-2 temperature module

Assign some tips to the deck and pick one up so that we can aspirate:

In [11]:
from pylabrobot.resources.opentrons import opentrons_96_tiprack_300ul

tips300 = opentrons_96_tiprack_300ul(name="tips")
lh.deck.assign_child_at_slot(tips300, slot=11)

In [12]:
await lh.pick_up_tips(tips300["A5"])

Access the temperature controller's tube rack with the `tube_rack` attribute.

In [13]:
await lh.aspirate(t.tube_rack["A1"], vols=[20])

In [14]:
await lh.aspirate(t.tube_rack["A6"], vols=[20])

Return the tips to the tip rack when you are done.

In [15]:
await lh.return_tips()